# Спринт 4 «Предварительная обработка данных»

# Проект «Исследование надёжности заёмщиков»

## Навигация

1. [Описание проекта](#Описание-проекта)
1. [Описание данных](#Описание-данных)
1. [Предобработка данных](#Предобработка-данных)
   - [Удаление пропусков](#Удаление-пропусков)
   - [Обработка аномальных значений](#Обработка-аномальных-значений)
   - [Удаление пропусков (продолжение)](#Удаление-пропусков-(продолжение))
   - [Изменение типов данных](#Изменение-типов-данных)
   - [Обработка дубликатов](#Обработка-дубликатов)
   - [Категоризация данных](#Категоризация-данных)
1. [Исследование данных](#Исследование-данных)
1. [Общий вывод](#Общий-вывод)

## Описание проекта

[к навигации](#Навигация)

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели _кредитного скоринга_ — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Описание данных

[к навигации](#Навигация)

- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

In [1]:
import os
import pandas as pd

In [2]:
data = pd.read_csv(os.path.join('..', 'datasets', 'data.csv'))
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

[к навигации](#Навигация)

### Удаление пропусков

[к навигации](#Навигация)

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
# для ответа на вопрос 3.5
question_3_5 = (data['days_employed'].isna() & data['total_income'].isna()).sum()

Столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

[к навигации](#Навигация)

Общий трудовой стаж в днях `days_employed` не может быть отрицательным значением.

In [7]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставляем их как есть. Тем более этот столбец не понадобится нам для исследования.

Выведем перечень уникальных значений столбца `children`.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения.

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

[к навигации](#Навигация)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

[к навигации](#Навигация)

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

[к навигации](#Навигация)

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [15]:
data['education'] = data['education'].str.lower()

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

### Категоризация данных

[к навигации](#Навигация)

Cоздадим столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных

[к навигации](#Навигация)

**Есть ли зависимость между количеством детей и возвратом кредита в срок?**

In [23]:
data.groupby('children')['debt'].mean()

children
0    0.075438
1    0.092346
2    0.094542
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64

In [24]:
data['children'].value_counts()

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод:**  
1. Группы с 4 и 5 детьми получились невероятно маленькими: не думаю, что статистика по ним вообще значима.
2. Остаются группа без детей, с 1 и 2 детьми. Здесь можно сказать, что наличие детей увеличивается вероятность не выплатить кредит на 2-2.5 процентных пункта.

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [25]:
data.groupby('family_status')['debt'].mean()

family_status
Не женат / не замужем    0.097639
в разводе                0.070648
вдовец / вдова           0.066246
гражданский брак         0.093130
женат / замужем          0.075606
Name: debt, dtype: float64

In [26]:
data['family_status'].value_counts()

женат / замужем          12261
гражданский брак          4134
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: family_status, dtype: int64

**Вывод:** 
Все группы (по-моему) достаточно большие, чтобы статистика по ним была значима.  
1. Лучше всех платят люди, потерявшие половинку.
2. Чуть хуже платят люди в разводе.
3. На третьем и четвёртом месте распожились люди в браке, и не расписанные пары.
4. Хуже ввсех платят одиночки.

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [27]:
data.groupby('total_income_category')['debt'].mean()

total_income_category
A    0.080000
B    0.070602
C    0.084982
D    0.060172
E    0.090909
Name: debt, dtype: float64

In [28]:
data['total_income_category'].value_counts()

C    15921
B     5014
D      349
A       25
E       22
Name: total_income_category, dtype: int64

**Вывод:** 
1. Группы A, D, E очень малы.
2. Разница между C и B заметна: целых 1.5 процентных пункта.

**Как разные цели кредита влияют на его возврат в срок?**

In [29]:
data.groupby('purpose_category')['debt'].mean()

purpose_category
операции с автомобилем      0.093480
операции с недвижимостью    0.072551
получение образования       0.092528
проведение свадьбы          0.079118
Name: debt, dtype: float64

In [30]:
data['purpose_category'].value_counts()

операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

**Вывод:** 
1. Группы достаточно большие.
2. Лучше всего обстоят дела у людей, взявших кридт на недвижимость. Удивительно, но люди, проводящие свадьбы тоже достаточно хорошо возврашают деньги. (Должно быть окупаются подарками). Хуже всего дела у людей, берущих кредит на автомобиль и образование.

Пропуски наблюдаются в столбцах, фиксирующих дни трудового стажа и уровень дохода. Причём пропуски наблюдаются в строчках одновременно.  
Должно быть это безработные.

Заполнить пропуски медианным значением — лучшее решение для количественных переменных, потому что могут быть значения, которые сильно выбиваются из "среднего" диапазона и при вычислении среднего арифметического сильно уводят его значение.

## Общий вывод

[к навигации](#Навигация)

1. Заёмщики с детьми просрачивают возвращение кредита чаще на 2-2.5 процентных пункта.
2. Семейное положение влияет на наличие долга.
3. Уровень дохода влияет: люди из группы B на 1.5 процентных пункта возвращают деньги чаще, чем из группы C.
4. Цели тоже влияют.